In [290]:
import numpy as np
import os 
import time 
import cv2 
import json 
import imutils
from tqdm import tqdm

In [291]:
#path_annotation = "/home/asilla/duongnh/datasets/CrownPose/CrowdPose/crowdpose_train.json"
path_annotation = "/home/asilla/duongnh/project/Analys_CrownPose/AddNose_CrownPose/Converted_Annotation_hrnet.json"
path_image = "/home/asilla/duongnh/datasets/CrownPose/images"

In [292]:
file_annotation = open(path_annotation, "r")
dataset = json.load(file_annotation)


In [293]:
print(dataset.keys())

dict_keys(['info', 'categories', 'images', 'annotations'])


In [294]:
info_dataset = dataset["info"]
categories_dataset = dataset["categories"]
images_dataset = dataset["images"]
annotations_dataset = dataset["annotations"]


In [295]:
print(info_dataset)

{'description': 'CrowdPose annotations modified by MMPose team.', 'year': 2020, 'date_created': '2020/07/01'}


In [296]:
print(categories_dataset)

[{'supercategory': 'person', 'id': 1, 'name': 'person', 'keypoints': ['nose', 'neck', 'right_shoulder', 'right_elbow', 'right_wrist', 'left_shoulder', 'left_elbow', 'left_wrist', 'right_hip', 'right_knee', 'right_ankle', 'left_hip', 'left_knee', 'left_ankle', 'centeroid'], 'skeleton': [[2, 1], [2, 3], [2, 6], [2, 15], [15, 3], [15, 6], [15, 9], [15, 12], [3, 4], [4, 5], [6, 7], [7, 8], [9, 10], [10, 11], [12, 13], [13, 14]]}]


In [297]:
print(images_dataset[0])

{'file_name': '103492.jpg', 'id': 103492, 'height': 429, 'width': 640, 'crowdIndex': 0.08}


In [298]:
print(annotations_dataset[0]["iscrowd"])

0


In [299]:
def check_not_visiable_keypoint(num_kp, l_kp):
    count_kp = 0
    for i in range(14):
        visible = l_kp[i * 3 + 2]
        if visible != 0:
            count_kp += 1
    if count_kp != num_kp:
        print(count_kp, num_kp)

In [300]:
font = cv2.FONT_HERSHEY_SIMPLEX
  
  
# fontScale
fontScale = 0.2
   
# Blue color in BGR
color = (0, 0, 255)
  
# Line thickness of 2 px
thickness = 2

In [301]:
def load_annotation(path_annotation):
    file_annotation = open(path_annotation,'r')
    json_data = json.load(file_annotation)
    l_image = json_data['images']
    l_annotations = json_data['annotations']
    l_bbox = []
    l_keypoint = []
    l_num_keypoint = []
    for annotation in l_annotations:
        image_id = annotation["image_id"]
        num_keypoints = annotation["num_keypoints"]
        print(num_keypoints)
        keypoint = annotation["keypoints"]
        bbox = annotation["bbox"]
        l_bbox.append(bbox)
        l_num_keypoint.append(num_keypoints)
        l_keypoint.append(keypoint)
    return l_bbox, l_keypoint, l_num_keypoint


def visualize_bbox(image, bbox,color):
    xtop, ytop, width, height = bbox
    image = cv2.rectangle(image, (int(xtop), int(ytop)),\
                            (int(xtop + width), int(ytop + height)), color,1)
    return image


def visualize_keypoint(image, keypoints, color, join_keypoint_categories, annotation):
    l_keypoint = []
    # if len(keypoints) == 45:
    #     confidence_score = annotation["confidence_score_nose"]
    #     range_data = 15
    #     x = int(keypoints[14 * 3])
    #     y = int(keypoints[14 * 3 + 1])
    #     image = cv2.putText(image, confidence_score, (x,y), font, 
    #                0.55, (0,0,255), 1, cv2.LINE_AA)
    #     image = cv2.circle(image, (5 + x,y), 5,color, -1)
    # else:
    #     range_data = 14
    for i in range(15):
        x = int(keypoints[i * 3])
        y = int(keypoints[i * 3 + 1])
        visible = int(keypoint[i * 3 + 2])
        l_keypoint.append((x,y, visible))
        if visible != 0:
            image = cv2.circle(image, (x,y), 2,color, -1)
    for pair_point in join_keypoint_categories:
        start_point_index, end_point_index = pair_point
        start_point_index = start_point_index - 1 
        end_point_index = end_point_index - 1
        if l_keypoint[start_point_index][2] != 0 and l_keypoint[end_point_index][2]:
            image = cv2.line(image, (l_keypoint[start_point_index][0], l_keypoint[start_point_index][1]),\
                                    (l_keypoint[end_point_index][0],l_keypoint[end_point_index][1]), color, 1)
    return image 

def check_size_bbox(bbox):
    threshold_small = 48 * 48 
    xtop, ytop, width, height = bbox
    size_bbox = width * height 
    if size_bbox < threshold_small:
        return "small"
    else:
        return "large"


def valid_num_keypoint(l_annotation):
    for annotation in l_annotation:
        num_keypoints = annotation["num_keypoints"]
        if num_keypoints == 0:
            return True
    return False



In [302]:
l_image = images_dataset
l_annotations = annotations_dataset
l_categories = categories_dataset
join_keypoint_categories = l_categories[0]["skeleton"]
print(join_keypoint_categories)

dict_image_name = {}
dict_image_id = {}

for info_image in l_image:
    image_id = info_image["id"]
    image_name = info_image["file_name"]
    dict_image_name[image_name] = image_id

for annotation in l_annotations:
    
    image_id = annotation["image_id"]
    num_keypoints = annotation["num_keypoints"]
    keypoint = annotation["keypoints"]
    bbox = annotation["bbox"]
    
    if image_id not in dict_image_id:
        dict_image_id[image_id] = []
        dict_image_id[image_id].append(annotation)
    else:
        dict_image_id[image_id].append(annotation)
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 2
color = (0, 0, 255)
thickness = 2
l_color_keypointjoint = [(255,0,0), (0,255,255), (255,0,255),(0,140,255),(32,165,218),(0,128,128),(0,100,0),(79,79,47),(204,209,72),(112,25,25),(139,0,139)]
count = 0
l_count_human = []
path_folder_source = "/home/asilla/duongnh/datasets/CrownPose/images"
path_visualize = "visualize_hrnet"
count_visualize = 0
for file in sorted(os.listdir(path_folder_source)):
    count_human = 0
    if file not in dict_image_name:
        continue 
    image_id = dict_image_name[file]
    l_annotation = dict_image_id[image_id]
    
    index_color_joint = 0
    image = cv2.imread(path_folder_source + "/" + file)
    for annotation in l_annotation:
        num_keypoints = annotation["num_keypoints"]
        keypoint = annotation["keypoints"]
        bbox = annotation["bbox"]
        red = (0,0,255)
        blue_light  = (255,255,0)
        purple = (239,73,249)
        orange = (54, 183, 242)
        green = (30,255, 78)
        check_not_visiable_keypoint(num_keypoints, keypoint)
        type_bbox = check_size_bbox(bbox)
       
        color_join = l_color_keypointjoint[index_color_joint]
        image = visualize_bbox(image, bbox, red)
        image = visualize_keypoint(image, keypoint, color_join, join_keypoint_categories, annotation)
        index_color_joint += 1
        if index_color_joint > len(l_color_keypointjoint) - 1:
            index_color_joint = 0
    count_visualize += 1
    if count_visualize > 50: 
        break
    cv2.imwrite(path_visualize + "/" + file, image)
    

[[2, 1], [2, 3], [2, 6], [2, 15], [15, 3], [15, 6], [15, 9], [15, 12], [3, 4], [4, 5], [6, 7], [7, 8], [9, 10], [10, 11], [12, 13], [13, 14]]
14 13


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Expected sequence length 2, got 3
>  - Can't parse 'pt1'. Expected sequence length 2, got 3


In [ ]:
print(l_count_human)

[]
